Multi-task recommenders

In [1]:
!pip install tf_keras
!pip install pymongo


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Mahesha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [8]:
ratings = tfds.load("movielens/latest-small-ratings", split="train")
movies = tfds.load('movielens/100k-movies', split="train")

In [9]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
movies = movies.map(lambda x: x["movie_title"])

In [10]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [12]:
import tensorflow_recommenders as tfrs

In [13]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("user_rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

---



Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [14]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [16]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 6s 335ms/step - root_mean_squared_error: 2.5079 - factorized_top_k/top_1_categorical_accuracy: 0.0554 - factorized_top_k/top_5_categorical_accuracy: 0.0737 - factorized_top_k/top_10_categorical_accuracy: 0.0825 - factorized_top_k/top_50_categorical_accuracy: 0.1193 - factorized_top_k/top_100_categorical_accuracy: 0.1498 - loss: 5.7151 - regularization_loss: 0.0000e+00 - total_loss: 5.7151
Epoch 2/3
10/10 [==============================] - 3s 324ms/step - root_mean_squared_error: 1.0607 - factorized_top_k/top_1_categorical_accuracy: 0.0443 - factorized_top_k/top_5_categorical_accuracy: 0.0549 - factorized_top_k/top_10_categorical_accuracy: 0.0693 - factorized_top_k/top_50_categorical_accuracy: 0.1014 - factorized_top_k/top_100_categorical_accuracy: 0.1392 - loss: 1.1190 - regularization_loss: 0.0000e+00 - total_loss: 1.1190
Epoch 3/3
5/5 [==============================] - 2s 166ms/step - root_mean_squared_error: 1.0221 - factorized_top_

The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [20]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [21]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 4s 344ms/step - root_mean_squared_error: 3.6501 - factorized_top_k/top_1_categorical_accuracy: 0.3976 - factorized_top_k/top_5_categorical_accuracy: 0.4201 - factorized_top_k/top_10_categorical_accuracy: 0.4344 - factorized_top_k/top_50_categorical_accuracy: 0.4803 - factorized_top_k/top_100_categorical_accuracy: 0.5110 - loss: 69724.9950 - regularization_loss: 0.0000e+00 - total_loss: 69724.9950
Epoch 2/3
10/10 [==============================] - 3s 348ms/step - root_mean_squared_error: 3.6356 - factorized_top_k/top_1_categorical_accuracy: 0.2654 - factorized_top_k/top_5_categorical_accuracy: 0.3449 - factorized_top_k/top_10_categorical_accuracy: 0.3668 - factorized_top_k/top_50_categorical_accuracy: 0.4604 - factorized_top_k/top_100_categorical_accuracy: 0.5235 - loss: 69084.8452 - regularization_loss: 0.0000e+00 - total_loss: 69084.8452
Epoch 3/3
5/5 [==============================] - 1s 167ms/step - root_mean_squared_error: 3.6371 -

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [22]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [23]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 4s 357ms/step - root_mean_squared_error: 2.8473 - factorized_top_k/top_1_categorical_accuracy: 0.3935 - factorized_top_k/top_5_categorical_accuracy: 0.4196 - factorized_top_k/top_10_categorical_accuracy: 0.4306 - factorized_top_k/top_50_categorical_accuracy: 0.4663 - factorized_top_k/top_100_categorical_accuracy: 0.4998 - loss: 69740.2926 - regularization_loss: 0.0000e+00 - total_loss: 69740.2926
Epoch 2/3
10/10 [==============================] - 4s 366ms/step - root_mean_squared_error: 1.3369 - factorized_top_k/top_1_categorical_accuracy: 0.2284 - factorized_top_k/top_5_categorical_accuracy: 0.3149 - factorized_top_k/top_10_categorical_accuracy: 0.3454 - factorized_top_k/top_50_categorical_accuracy: 0.4574 - factorized_top_k/top_100_categorical_accuracy: 0.5258 - loss: 69053.3551 - regularization_loss: 0.0000e+00 - total_loss: 69053.3551
Epoch 3/3
5/5 [==============================] - 1s 174ms/step - root_mean_squared_error: 1.0211 -

The result is a model that performs roughly as well on both tasks as each specialized model.

### Making prediction

We can use the trained multitask model to get trained user and movie embeddings, as well as the predicted rating:

In [24]:
trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array(["Dances with Wolves (1990)"])
  })
print("Predicted rating:")
print(predicted_rating)

Predicted rating:
tf.Tensor([[3.5474572]], shape=(1, 1), dtype=float32)


While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).